Paquetes necesarios

In [1]:
import cv2  
import math 
from ultralytics import YOLO

Trabajo de curso 

In [13]:
model = YOLO('yolov8n.pt')

# Nombre de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "_", "bus", "_", "truck", "_",
              "_", "_", "_", "_", "_", "bird", "cat",
              "dog", "horse", "_", "_", "_", "_", "_", "_", "_", "umbrella",
              "_", "_", "_", "_", "_", "_", "_", "_", "_",
              "_", "_", "_", "_", "_", "_", "_",
              "_", "_", "_", "_", "_", "_", "_", "_", "_",
              "_", "_", "_", "_", "_", "_", "_", "_", "_",
              "_", "_", "_", "_", "_", "_", "_", "_",
              "_", "_", "_", "_", "_", "_", "_", "_", "_",
              "_", "_", "_"
              ]


vid = cv2.VideoCapture("TUD-Crossing-det.webm")
#Cuadrado resultado
result_path = "white.jpg"
result_image = cv2.imread(result_path)
# Lista de los posibles obstáculos en orden de prioridad
priority_list = ["truck","bus","car","motorbike","bicycle","person","umbrella","horse","dog","cat","bird"]
#Contador para suavizar la imagen resultado
counter = 10
while(True):
    ret, img = vid.read()
    if ret:   
        results = model(img, stream=True)
        
        # Para cada detección
        for r in results:
            boxes = r.boxes
            coincidencies = []
            
            for box in boxes:
                # Filtro por clase
                cls = int(box.cls[0])
                if(classNames[cls] == "_"):
                    continue
                # Filtro por confianza
                confidence = math.ceil((box.conf[0]*100))/100
                if(confidence < 0.3):
                    continue

                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                #Mostrar confianza
                #print("Confidence --->",confidence)

                #Mostrar clase
                #print("Class name -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)
                # Guarda el objeto detectado y su confianza en un array
                coincidencies.append((confidence,classNames[cls]))
            
            # Este es el umbral para la confianza de la detección (de 0 a 1)
            umbral = 0.8
            # Método para cambiar cúal es el objeto más prioritario de la imagen 
            def update_priority(detected_word, current_priority):
                if current_priority[1] == "nada":
                    return detected_word
                # prioridad = nueva palabra
                if (current_priority[0] >= umbral and detected_word[0] >= umbral) or (current_priority[0] < umbral and detected_word[0] < umbral):
                    word_index = priority_list.index(detected_word[1])
                    if word_index < priority_list.index(current_priority[1]):
                        return detected_word
                    return current_priority
                # prioridad = umbral< y nueva palabra = umbral+
                elif current_priority[0] < umbral and detected_word[0] >= umbral:
                    return detected_word
                # prioridad = umbral+ y nueva palabra = umbral<
                elif current_priority[0] >= umbral and detected_word[0] < umbral:
                    return current_priority
                
            
            #print("Las coincidencias en esta iteración son: ",coincidencies)
            
            # Equivalente a no haber nada en la imagen que detectar
            prioridad = (0,"nada")

            for coincidence in coincidencies:
                for obstacle in priority_list:
                    if(obstacle == coincidence[1]):
                        prioridad = update_priority(coincidence,prioridad)
                
            if prioridad[1] == "nada":
                text1 = "There is nothing on the other side."
                text2 = ""
                text3 = "You can continue carefully."
            else:
                text1 = "Be careful! There is a " + str(prioridad[0]) + " chance of a:"
                text2 = prioridad[1]
                text3 = " on the other side."
            # Escribe en el rectángulo que hay al otro lado
            if counter == 10: 
                result_image = cv2.imread(result_path)
                Red = prioridad[0] * 255
                cv2.putText(result_image, text1 , [50, 100], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, Red), 2)
                cv2.putText(result_image, text2 , [50, 180], cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, Red), 4)
                cv2.putText(result_image, text3 , [50, 250], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, Red), 2)

                # Añadimos el obstáculo detectado al rectángulo resultado
                new_img = img[y1:y2,x1:x2]
                new_img = cv2.resize(new_img,(300,220))
                height, width, channels = new_img.shape
                result_image[200:200+height,400:400+width] = new_img
                cv2.imshow("Imagen",result_image)
                counter = 0
            else:
                counter = counter + 1

            # Se vacía al final para que se pueda actualizar correctamente
            coincidencies = []

    if cv2.waitKey(20) == 27:
        break
cv2.destroyAllWindows()






0: 480x640 6 persons, 1 car, 116.1ms
Speed: 3.0ms preprocess, 116.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 persons, 2 cars, 121.3ms
Speed: 2.0ms preprocess, 121.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 3 cars, 1 truck, 119.3ms
Speed: 2.0ms preprocess, 119.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 4 cars, 113.4ms
Speed: 2.0ms preprocess, 113.4ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 persons, 1 backpack, 115.1ms
Speed: 1.0ms preprocess, 115.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 1 car, 1 handbag, 110.5ms
Speed: 3.0ms preprocess, 110.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 112.1ms
Speed: 1.0ms preprocess, 112.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 6 persons, 114.